In [1]:
import numpy as np
import pandas as pd
import json
import time
import twitter
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor, Json
from datetime import datetime

/Applications/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
%run sql_pw.py

In [3]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    ''' 
    Returns both a connection and a cursor object for your database
    '''

    con = pg2.connect(host=IP_ADDRESS, #allows you to navigate db
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    '''
    Executes a query directly to a database, without having to create a cursor and connection each time. 
    '''
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

def insert_entry_json(data, tablename=None):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
    con.commit()
    con.close()

In [4]:
twitter_keys = {
    'consumer_key':        'OoTmt8ytXol9yRJxLvBbYER5e',
    'consumer_secret':     'wHEGZ4juW2AIUU1GFp32oA8vGtre8Se5hSPp5DfOmeNJ2a3Ld7',
    'access_token_key':    '1080874755215872001-9F7ZdDBK2b9hgBKjSTiTQXIWY2XdyM',
    'access_token_secret': 'HIMPWRosWf2Wlu73wEPWjsmjobO3FvtRaa3MneDW0wT3q'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
)

In [5]:
def streamTweets(city, geocode, term, before, count, sql_db):
    for i in range(1,8):
        year, month, day = before.split('-')
        day = int(day)
        day-=1
        day = str(day).zfill(2)
        date = year + month + day
        after = datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
        
        results = api.GetSearch(
            term = term,
            geocode = geocode,
            return_json = True
        )

        insert_entry_json(results['statuses'], sql_db)
        before = after

In [6]:
def tweet_repeater(city, geocode, term, start, repeats=15, count=100, sql_db='raw_tweets'):
    for i in range(repeats):
        before = start
        
        streamTweets(city, geocode, term, before, count, sql_db)
        print(f'Loop {i+1} complete. Raw tweets pushed to {sql_db}.')
        time.sleep(40)
        
    print('All tweets pulled.')

In [7]:
tweet_repeater(city='cha', 
               geocode='35.2271,-80.8431,10mi', 
               term='justin timberlake tour',
               start='2019-01-08', 
               repeats=100, 
               count=100, 
               sql_db='raw_tweets')

Loop 1 complete. Raw tweets pushed to raw_tweets.
Loop 2 complete. Raw tweets pushed to raw_tweets.
Loop 3 complete. Raw tweets pushed to raw_tweets.
Loop 4 complete. Raw tweets pushed to raw_tweets.
Loop 5 complete. Raw tweets pushed to raw_tweets.
Loop 6 complete. Raw tweets pushed to raw_tweets.
Loop 7 complete. Raw tweets pushed to raw_tweets.
Loop 8 complete. Raw tweets pushed to raw_tweets.
Loop 9 complete. Raw tweets pushed to raw_tweets.
Loop 10 complete. Raw tweets pushed to raw_tweets.
Loop 11 complete. Raw tweets pushed to raw_tweets.
Loop 12 complete. Raw tweets pushed to raw_tweets.
Loop 13 complete. Raw tweets pushed to raw_tweets.
Loop 14 complete. Raw tweets pushed to raw_tweets.
Loop 15 complete. Raw tweets pushed to raw_tweets.
Loop 16 complete. Raw tweets pushed to raw_tweets.
Loop 17 complete. Raw tweets pushed to raw_tweets.
Loop 18 complete. Raw tweets pushed to raw_tweets.
Loop 19 complete. Raw tweets pushed to raw_tweets.
Loop 20 complete. Raw tweets pushed to r

In [8]:
searchterms = [
    'jt', 'justin timberlake', 'jtimberlake', 'justin', 'timberlake', 'concert', 'show', 'mirror', 'cry me a river',
    'man of the woods', 'tour', 'music', 'PNC', 'welcome back', 'TN kids', 'tennessee kids'
]

In [9]:
# cities_dict = {'cha': {'geocode':'35.2271,-80.8431,2mi', 'start_date':'2019-01-08'}, 
#                'atl': {'geocode':'33.753746,-84.386330,2mi', 'start_date':'2019-01-10'},
#                'mem': {'geocode':'35.14953,-90.04898,2mi', 'start_date':'2019-01-12'},
#                'no': {'geocode':'29.951065,-90.071533,2mi', 'start_date':'2019-01-15'},
#                'nlr': {'geocode':'34.746483,-92.289597,2mi', 'start_date':'2019-01-17'}}

In [10]:
# def streamTweets(city, count):
#     i=0
#     while i < 100:
#         results = api.GetSearch(
#             geocode = cities_dict[city]['geocode'],
#             since = cities_dict[city]['start_date'],
#             return_json = True
#             )
#         i+=1
#         time.sleep(1)

In [11]:
# streamTweets(city='cha', count=100)